### Automatic model selection

Test notebook for automatic model selection.

In [1]:
import datetime
import logging
import sys
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
import keras.backend as K

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

import ann_framework.aux_functions as aux_functions

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
import fetch_to_keras
#from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression, SequenceTunableModelClassification

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPSS import CMAPSSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
from ann_framework.data_handlers.data_handler_CIFAR10 import CIFAR10DataHandler
#from data_handler_MNIST import MNISTDataHandler
#from data_handler_CMAPS import CMAPSDataHandler

size_scaler = 1

Using TensorFlow backend.


### Given a model, get the compiled model

In [2]:
def get_compiled_model(model, problem_type, optimizer_params=[]):
    """Obtain a keras compiled model"""
    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001, beta_1=0.5)
    
    if problem_type == 1:
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    
    return model


def create_tunable_model(model_genotype, problem_type, input_shape, data_handler, model_number):
    
    K.clear_session()
    
    model = fetch_to_keras.decode_genotype(model_genotype, problem_type, input_shape, 1)
    
    model = get_compiled_model(model, problem_type, optimizer_params=[])
    
    if problem_type == 1:
        tModel = SequenceTunableModelRegression('ModelReg_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
    else:
        tModel = SequenceTunableModelClassification('ModelClass_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
        
    return tModel

### Load cmaps data handler

In [3]:
def cmaps_dhandler():

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 25
    window_stride = 1
    max_rul = 130

    dHandler_cmaps = CMAPSSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)

    input_shape = (len(selected_features)*window_size, )

    return dHandler_cmaps, input_shape

### Function to save top models

In [4]:
def save_best_models(best_models_list, global_best_model_index, saveto, input_shape, data_handler, 
                     problem_type=1, data_scaler=None, train_epochs=100, metrics=[], round=0):
    
    n_models = len(best_models_list)
    
    for ind_model, i in zip(best_models_list, range(n_models)):
        
        tModel = create_tunable_model(ind_model.stringModel, problem_type, input_shape, data_handler, i)
        kmodel = tModel.model
        model_json = kmodel.to_json()
        
        #Save model's architecture
        string_append = str(i) if i != global_best_model_index else 'global'
        with open(saveto+"bestModel_"+string_append+".json", "w") as json_file:
            json_file.write(model_json)
            
    #Train the global best, model has to be recompiled
    ind_model = best_models_list[global_best_model_index]
    tModel = create_tunable_model(ind_model.stringModel, problem_type, input_shape, data_handler, n_models)
    
    print(tModel.model.summary())
    print(tModel.data_handler)
    
    if tModel.data_handler.data_scaler != None:
        print("Using data handler scaler")
    elif tModel.data_scaler != None:
        print("Using tModel scaler (Overriding data handler scaler)")
    else:
        print("No data scaling used")
    
    if data_scaler != None:
        tModel.data_handler.data_scaler = None
        tModel.data_scaler = data_scaler
        
    tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
    tModel.print_data()
    tModel.epochs = train_epochs

    tModel.train_model(verbose=1)
    
    tModel.evaluate_model(metrics, round=round)
    
    kmodel = tModel.model
            
    # serialize weights to HDF5
    kmodel.save_weights(saveto+"bestModel_global.h5")
    
    print("Saved models for dataset 1 to disk")
        

### Test on MNIST

In [ ]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (784,)
pop_size = 5
tournament_size = 3
max_similar = 3
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None
global_best_index = 0

learningRate_scheduler = LearningRateScheduler(aux_functions.step_decay)

scaler = None

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_mnist_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#mnist datahandler
dHandler_mnist = MNISTDataHandler()

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, 
                       tournament_size, max_similar, epochs=5, cross_val=0.2, size_scaler=size_scaler,
                       max_generations=10, binary_selection=True, mutation_ratio=0.4, 
                       similarity_threshold=0.2, more_layers_prob=0.8)

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    
    best = automatic_model_selection.run_experiment(config, dHandler_mnist, count_experiments + 1, unroll=unroll,
                                                    learningRate_scheduler=learningRate_scheduler, 
                                                    tModel_scaler=scaler, verbose_data=0)

    best.individual_label = count_experiments
    
    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best
            global_best_index = count_experiments

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
automatic_model_selection.print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

Launching experiment 1

Generation 1
launch new
True
gen similar
False
Fetching model 0 to keras
Evaluating model 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 48)                37680     
_________________________________________________________________
dropout_1 (Dropout)          (None, 48)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 384)               18816     
_________________________________________________________________
dropout_2 (Dropout)          (None, 384)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 432)               166320    
_________________________________________________________________
dropout_3 (Dropout)          (None, 432)               0         
__________________________

/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 2s 32us/step - loss: 1.7975 - acc: 0.2584 - val_loss: 1.5600 - val_acc: 0.3288
Epoch 2/5
48000/48000 [==============================] - 1s 10us/step - loss: 1.1434 - acc: 0.5072 - val_loss: 0.9704 - val_acc: 0.5882
Epoch 3/5
48000/48000 [==============================] - 1s 11us/step - loss: 0.7728 - acc: 0.6778 - val_loss: 0.6210 - val_acc: 0.7313
Epoch 4/5
48000/48000 [==============================] - 1s 11us/step - loss: 0.6493 - acc: 0.7291 - val_loss: 0.5870 - val_acc: 0.7595
Epoch 5/5
12000/12000 [==============================] - 0s 19us/step
Fetching model 1 to keras
Evaluating model 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 904)               709640    
_________________________________________________________________
dropout_1 

48000/48000 [==============================] - 1s 13us/step - loss: 0.6181 - acc: 0.7973 - val_loss: 0.2331 - val_acc: 0.9279
Epoch 2/5
48000/48000 [==============================] - 0s 7us/step - loss: 0.2302 - acc: 0.9298 - val_loss: 0.1486 - val_acc: 0.9537
Epoch 3/5
48000/48000 [==============================] - 0s 7us/step - loss: 0.1700 - acc: 0.9482 - val_loss: 0.1266 - val_acc: 0.9615
Epoch 4/5
48000/48000 [==============================] - 0s 7us/step - loss: 0.1402 - acc: 0.9569 - val_loss: 0.1114 - val_acc: 0.9658
Epoch 5/5
12000/12000 [==============================] - 0s 15us/step

Generating offsprings
Applying Mutation
Launch new generation?: True

Generation 2
launch new
True
gen similar
False
Fetching model 0 to keras
Evaluating model 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 24)                18840     
______________________________________

In [ ]:
save_best_models(global_best_list, global_best_index, 'best_models/mnist/alpha{}/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dHandler_mnist, problem_type=problem_type, train_epochs=100)

### Test on CIFAR10

In [ ]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (3072,)
pop_size = 5
tournament_size = 3
max_similar = 3
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None
global_best_index = 0

scaler = None

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_cifar10_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#mnist datahandler
dHandler_cifar = CIFAR10DataHandler()

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, max_similar, epochs=5, cross_val=0.2, size_scaler=0.4,
                       max_generations=10, binary_selection=True, mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    
    best = automatic_model_selection.run_experiment(config, dHandler_cifar, count_experiments + 1, unroll=unroll,
                                                    learningRate_scheduler=learningRate_scheduler, 
                                                    tModel_scaler=scaler, verbose_data=0)
    
    best.individual_label = count_experiments

    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best
            global_best_index = count_experiments

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
automatic_model_selection.print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

In [ ]:
save_best_models(global_best_list, global_best_index, 'best_models/cifar10/alpha{}/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dHandler_cifar, problem_type=problem_type, train_epochs=100)

### Test on CMAPSS

In [ ]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 1  #1 for regression, 2 for classification
output_shape = 1 #If regression applies, number of classes
input_shape = None
pop_size = 5
tournament_size = 3
max_similar = 3
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None
global_best_index = 0

scaler = MinMaxScaler(feature_range=(-1, 1))

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_cmaps_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#cmaps datahandler
dhandler_cmaps, input_shape = cmaps_dhandler()
print(input_shape)

#mnist datahandler
#dHandler_mnist = MNISTDataHandler()

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, max_similar, epochs=5, cross_val=0.2, size_scaler=0.4,
                       max_generations=10, binary_selection=True, mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    
    best = automatic_model_selection.run_experiment(config, dhandler_cmaps, count_experiments + 1, unroll=unroll,
                                                    learningRate_scheduler=learningRate_scheduler, 
                                                    tModel_scaler=scaler, verbose_data=0)
    
    best.individual_label = count_experiments

    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best
            global_best_index = count_experiments

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
automatic_model_selection.print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

In [ ]:
save_best_models(global_best_list, global_best_index, 'best_models/cmapss/alpha{}/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dhandler_cmaps, problem_type=problem_type, train_epochs=100, 
                 data_scaler=scaler, metrics=['rhs', 'rmse'], round=2)